In [3]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.cluster import KMeans
import re
import preprocessor as p#forming a separate feature for cleaned tweets
import string
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import adjusted_rand_score
from sklearn.feature_extraction.text import TfidfVectorizer
import glob

In [2]:
path = "Tweet-Data"

In [5]:
files = glob.glob(path + "/*jl")

In [9]:
data = []
for filename in files:
    data.append(open(filename))

In [10]:
doc = []
for file in data:
    for line in file:
        doc.append(json.loads(line))

In [16]:
len(doc)

1970

In [17]:
emptyList = []
for dict in doc:
    emptyList.append(dict['response'])
    

In [18]:
def check(dictionary):
    if dictionary['result_count'] == 0: return False
    else: return True

In [19]:
emptyData = []
for dict in emptyList:
    if check(dict['meta']): emptyData.append(dict['data'])

In [21]:
df = pd.DataFrame(emptyData[0])

In [22]:
for obj in range(1,len(emptyData)):
    df = df.append(pd.DataFrame(emptyData[obj]))

In [23]:
df.shape

(855983, 17)

In [24]:
df.to_pickle("../pickles/all_data_frame.plk")

In [29]:
df_clean = df.copy()

In [30]:
df_clean = df_clean[['lang','author_id','source','text', 'created_at']]

In [31]:
def isRT(text):
        if text.startswith("RT"): return "True"
        else: return "False"

In [32]:
df_clean['is_RT'] = df_clean['text'].map(isRT)

In [33]:
df_clean = df_clean[df_clean['is_RT']=="False"]

In [34]:
df_clean = df_clean[(df_clean['lang'] == "en")]

In [35]:
def createdAt(date):
    return date.split('-')[:1][0]
    

In [36]:
df_clean['year'] = df_clean.created_at.map(createdAt)

In [37]:
df_clean = df_clean.drop(columns=['created_at', 'lang'], axis=1)

In [38]:
df_clean['hashtag'] = df_clean['text'].apply(lambda x: re.findall(r"#(\w+)", x))

In [39]:
def txt_cleaner(text):
    
    text = text.encode(encoding="ascii", errors="ignore")
    text = text.decode() # removing unicode from the text 
    
    text = p.clean(text)
    
    text = text.lower() #lowering all the text
    
    punct = set(string.punctuation)
    text = "".join([ch for ch in text if ch not in punct]) #remove punctation
    
    stop_words = set(stopwords.words("english"))
    text = " ".join([word for word in text.split() if word not in stop_words]) #remove stopwords
    # try out stemming and lemmatization
    return text

In [40]:
df_clean['text'] = df_clean.text.map(txt_cleaner)

In [41]:
df_clean = df_clean.drop_duplicates(subset=['text'])

In [42]:
df_clean.to_pickle("../pickles/clean_all.plk")

In [44]:
df_clean.shape

(176533, 6)